## Plots of SST for S-MODE region


In [1]:
cd C:\users\jtomf\Documents\Python\S-MODE_analysis\code

[WinError 3] The system cannot find the path specified: 'C:\\users\\jtomf\\Documents\\Python\\S-MODE_analysis\\code'
C:\Users\Home\Documents\GitHub\OpenCTD_testing\code


In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime

import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
# For great circle distance between two points, use gsw.geostrophy.distance(lon, lat, p=0, axis=-1)

import functions  # requires functions.py from this repository

ModuleNotFoundError: No module named 'xarray'

In [ ]:
# %matplotlib inline
%matplotlib qt5
plt.rcParams['figure.figsize'] = (6,5)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 700
plt.close('all')

__figdir__ = '../plots/' + 'SMODE_'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [ ]:
savefig = True
zoom = True
if zoom:
    xmin, xmax = (-126,-122)
    ymin, ymax = (36,39)
    levels = np.linspace(13.5,15.5,21)+.5
    zoom_str='zoom'
else:
    xmin, xmax = (-126,-121)
    ymin, ymax = (35, 39)
    levels = np.linspace(12.5,16,21)+0
    zoom_str='wide'
    

In [ ]:
def plot_ops_area(ax,**kwargs):
    """ Add polygon to show S-MODE pilot operations area
         Inputs
            - matplotlib.pyplot.plot kwargs

         return
            - exit code (True if OK)

    """

    # Add S-MODE pilot operations area
    ''' 

    New corners of pentagon:
    38° 05.500’ N, 125° 22.067’ W
    37° 43.000’ N, 124° 00.067’ W
    37° 45.000’ N, 123° 26.000‘ W
    36° 58.000’ N, 122° 57.000’ W
    36° 20.000’ N, 124° 19.067’ W 
    '''
    
    coord = [[-(125+22.067/60),38+5.5/60], [-(124+0.067/60),37+43/60], [-(123+26/60),37+45/60], [-(122+57/60),36+58/60], [-(124+19.067/60),36+20/60]]
    coord.append(coord[0]) #repeat the first point to create a 'closed loop'

    xs, ys = zip(*coord) #create lists of x and y values

    if ax is None:
        ax = plt.gca()    
    # ax.plot(xs,ys,transform=ccrs.PlateCarree()) 
    ax.plot(xs,ys,**kwargs) 
    
    SF_lon=-(122+25/60)
    SF_lat= 37+47/60
    
    # mark a known place to help us geo-locate ourselves
    ax.plot(SF_lon, SF_lat, 'o', markersize=3, zorder=10, **kwargs)
    ax.text(SF_lon-5/60, SF_lat+5/60, 'San Francisco', fontsize=8, zorder=10, **kwargs)
    # ax.text(np.mean(xs)-.6, np.mean(ys)-.3, 'S-MODE ops area', fontsize=8, **kwargs)
    print(kwargs)
    
    return(xs,ys,ax)


In [ ]:
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPB/AVHRR_METOPB_20210930T060000Z.nc'
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/MODIS_Terra/MODIS_Terra_20210930T065001Z.nc'3 
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20210929T213000Z.nc'  #This one is good

#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211016T095000Z.nc' # good one
# url = 'http://smode.whoi.edu:8080/thredds/fileServer/satellite/AVHRR_METOPA/AVHRR_METOPA_20211019T031000Z.nc#bytes' #Really great!
# url = 'http://smode.whoi.edu:8080/thredds/fileServer/satellite/VIIRS_NRT/VIIRS_NRT_20211019T103001Z.nc#mode=bytes' #Not awesome, but coincident with B200 flight on 10/19
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/MODIS_Terra/MODIS_Terra_20211011T233459Z.nc'
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211022T111000Z.nc'
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPA/AVHRR_METOPA_20211022T161000Z.nc' # good one!

#Best sequence
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPC/AVHRR_METOPC_20211019T052000Z.nc'  # prettiest of all!!
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211025T214001Z.nc'

# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPC/AVHRR_METOPC_20211025T050000Z.nc' #not good, but looking for front...
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211025T102000Z.nc'
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211025T214001Z.nc'

#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPA/AVHRR_METOPA_20211028T033000Z.nc'
url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211104T103001Z.nc'
ds = xr.open_dataset(url)


In [ ]:
ds

In [ ]:
fig = plt.figure()
ax = plt.axes(projection = ccrs.PlateCarree(central_longitude=200))  # Orthographic
extent = [xmin, xmax, ymin, ymax]
ax.set_extent(extent, crs=ccrs.PlateCarree())
'''daystr=ds.time.dt.day.astype(str).values[0]
monstr=ds.time.dt.month.astype(str).values[0]
yrstr=ds.time.dt.year.astype(str).values[0]
day_str = monstr+'-'+daystr+'-'+yrstr
'''
day_str=ds.time.dt.strftime("%a, %b %d %H:%M").values[0]
day_str2=ds.time.dt.strftime("%m-%d-%Y").values[0]

ax.set_title('SST, ' + day_str, size = 10.)

#plt.set_cmap(cmap=plt.get_cmap('nipy_spectral'))
plt.set_cmap(cmap=plt.get_cmap('turbo'))
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.ylabels_right = False
#gl.xlocator = matplotlib.ticker.MaxNLocator(10)
#gl.xlocator = matplotlib.ticker.AutoLocator
# gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(0, 360 ,30))

ax.coastlines()
ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.6,.6,.6], edgecolor='black')
# cs = ax.contourf(ds.lon,ds.lat,np.squeeze(ds.sea_surface_temperature)-273.15, levels, extend='both', transform=ccrs.PlateCarree())
cs = ax.pcolormesh(ds.lon,ds.lat,np.squeeze(ds.sea_surface_temperature)-273.15, vmin=levels[0], vmax=levels[-1], transform=ccrs.PlateCarree())
# cb = plt.colorbar(cs,ax=ax,shrink=.8,pad=.05)
cb = plt.colorbar(cs,fraction = 0.022,extend='both')
cb.set_label('SST [$\circ$C]',fontsize = 10)
plot_ops_area(ax,transform=ccrs.PlateCarree(),color='w')

# Add a 10 km scale bar
km_per_deg_lat=gsw.geostrophy.distance((125,125), (37,38))/1000
deg_lat_equal_10km=10/km_per_deg_lat
x0 = -125
y0 = 37.75
ax.plot(x0+np.asarray([0, 0]),y0+np.asarray([0.,deg_lat_equal_10km]),transform=ccrs.PlateCarree(),color='k')
ax.text(x0+2/60, y0-.5/60, '10 km', fontsize=6,transform=ccrs.PlateCarree())



In [ ]:
# saildrones.plot('longitude','latitude','scatter',markersize=3,transform=ccrs.PlateCarree())
#ax.plot(saildrones['longitude'],saildrones['latitude'],'ko',markersize=3,transform=ccrs.PlateCarree())
saildrones=functions.get_current_position('saildrone')#,'ko', markersize=3,transform=ccrs.PlateCarree())
ax.plot(saildrones['longitude'],saildrones['latitude'],'ko',markersize=1,transform=ccrs.PlateCarree())
wg=functions.get_current_position('waveglider')#,'ko', markersize=3,transform=ccrs.PlateCarree())
hWG = ax.plot(wg['longitude'],wg['latitude'],'mo',markersize=1,transform=ccrs.PlateCarree())


In [ ]:
if savefig:
    plt.savefig(__figdir__+'SST_' + day_str2 + '_' + zoom_str + 'pretty2' +'.' +plotfiletype,**savefig_args)

In [ ]:
pwd